In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import time

ip=input("enter the Job title: ")
location=input("enter the location: ")
# Path to your chromedriver
chromedriver_path = "C:/Windows/chromedriver.exe"

# Setting up the webdriver
service = Service(chromedriver_path)
driver = webdriver.Chrome()

# Your Naukri login credentials
email = "nasirali.proitbridge@gmail.com"  # Replace with your email
password = "nasirnaukri123"  # Replace with your password

try:
    # Open the Naukri login page
    print("Opening the Naukri login page...")
    driver.get("https://www.naukri.com/nlogin/login")

    # Wait for the login page to load and enter email
    email_input = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.ID, "usernameField"))
    )
    email_input.clear()
    email_input.send_keys(email)

    # Enter password
    password_input = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.ID, "passwordField"))
    )
    password_input.clear()
    password_input.send_keys(password)

    # Click the login button
    login_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Login')]"))
    )
    login_button.click()

    print("Logging in...")

    # Wait for the homepage to load after login
    WebDriverWait(driver, 30).until(
        EC.url_contains("https://www.naukri.com/mnjuser/homepage")
    )

    print("Login successful, proceeding with the filter application...")

    # Retry locating the filter tab if a StaleElementReferenceException occurs
    filter_tab = None
    attempts = 0
    while attempts < 3:
        try:
            print("Waiting for the filter tab...")
            filter_tab = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div[1]/div"))
            )
            print("Filter tab found.")
            filter_tab.click()
            break  # Exit the loop if successful
        except StaleElementReferenceException:
            attempts += 1
            print(f"StaleElementReferenceException caught. Retrying... (Attempt {attempts})")
            time.sleep(2)  # Small delay before retrying

    if filter_tab is None:
        print("Failed to locate the filter tab after multiple attempts.")
    else:
        # Wait for the job title input field to be visible
        print("Waiting for the job title input field...")
        job_title_input = WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[1]/div/div/div[2]/div/div/div/input"))
        )
        print("Job title input field found.")
        job_title_input.clear()
        job_title_input.send_keys(ip)  # Replace with your desired job title
        time.sleep(2)  # Small delay to allow for auto-suggestions if any

        # Wait for the experience dropdown to be visible
        print("Waiting for the experience dropdown...")
        experience_dropdown = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div[1]/div/div/div[4]/div/div/input"))
        )
        print("Experience dropdown found.")
        experience_dropdown.click()

        # Select the "Fresher (Less than 1 year)" option
        print("Selecting 'Fresher (Less than 1 year)' option...")
        fresher_option = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "//li[@value='a0'][@title='Fresher']"))
        )
        fresher_option.click()

        # Wait for the location input field to be visible
        print("Waiting for the location input field...")
        location_input = WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[1]/div/div/div[6]/div/div/div/input"))
        )
        print("Location input field found.")
        location_input.clear()
        location_input.send_keys(location)  # Replace with your desired location
        time.sleep(2)  # Small delay to allow for auto-suggestions if any

        # Click the search button
        print("Waiting for the search button...")
        search_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div[1]/div/button/span[1]"))
        )
        print("Search button found.")
        search_button.click()

        print("Filters applied and search initiated successfully!")

        # Initialize lists to collect data
        titles = []
        companies = []
        experiences = []
        salaries = []
        locations = []
        skills_list = []
        posting_dates = []
        job_descriptions = []
        job_urls = []

        # Loop to iterate through the pages by constructing the page URL
        
        first_page_url = driver.current_url
        print(f"First page URL: {first_page_url}")

    # Logic to extract the base URL and include `-{i}?k`
        base_url = first_page_url.split("?")[0]
        query_params = first_page_url.split("?")[1]

        num_pages = 25

        for i in range(1, num_pages + 1):
            page_url = f"{base_url}-{i}?"
            print(f"Scraping page {i}: {page_url}")

        # Open the page
            driver.get(page_url)
            time.sleep(5)  # Wait for the page to load

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'lxml')
            postings = soup.find_all('div', class_='srp-jobtuple-wrapper')

            for post in postings:
                title = post.find('a', class_="title").text.strip() if post.find('a', class_="title") else 'N/A'
                company = post.find('a', class_="comp-name").text.strip() if post.find('a', class_="comp-name") else 'N/A'
                experience = post.find('span', class_="exp-wrap").text.strip() if post.find('span', class_="exp-wrap") else 'N/A'
                salary = post.find('span', class_="sal-wrap ver-line").text.strip() if post.find('span', class_="sal-wrap ver-line") else 'N/A'
                location = post.find('span', class_="loc-wrap ver-line").text.strip() if post.find('span', class_="loc-wrap ver-line") else 'N/A'
                skills = ', '.join([skill.text.strip() for skill in post.find('ul', class_="tags-gt")]) if post.find('ul', class_="tags-gt") else 'N/A'
                posting_date = post.find('span', class_="job-post-day").text.strip() if post.find('span', class_="job-post-day") else 'N/A'
                job_url = post.find('a', class_="title")['href'] if post.find('a', class_="title") else 'N/A'

                # Access the job description page to scrape the job description
                if job_url != 'N/A':
                    driver.get(job_url)
                    try:
                        # Use WebDriverWait to wait for the job description to be visible
                        job_description_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "dang-inner-html")]'))
                        )
                        job_description = job_description_element.text.strip()
                    except TimeoutException:
                        job_description = 'N/A'
                else:
                    job_description = 'N/A'

                # Append data to lists
                titles.append(title)
                companies.append(company)
                experiences.append(experience)
                salaries.append(salary)
                locations.append(location)
                skills_list.append(skills)
                posting_dates.append(posting_date)
                job_descriptions.append(job_description)
                job_urls.append(job_url)

finally:
    print("Closing the browser...")
    driver.quit()

# Create a DataFrame from the collected data
df = pd.DataFrame({
    'Title': titles,
    'Company': companies,
    'Experience': experiences,
    'Salary': salaries,
    'Location': locations,
    'Skills': skills_list,
    'Posting_Date': posting_dates,
    'Job_Description': job_descriptions,
    'Job_URL': job_urls
})

# Print the DataFrame
print("Total job postings found:", len(df))
print(df.head())

# Save DataFrame to CSV
df.to_csv('Job.csv', index=False)

print("DataFrame saved to 'output.csv'")

Opening the Naukri login page...
Logging in...
Login successful, proceeding with the filter application...
Waiting for the filter tab...
Filter tab found.
Waiting for the job title input field...
Job title input field found.
Waiting for the experience dropdown...
Experience dropdown found.
Selecting 'Fresher (Less than 1 year)' option...
Waiting for the location input field...
Location input field found.
Waiting for the search button...
Search button found.
Filters applied and search initiated successfully!
First page URL: https://www.naukri.com/data-scientist-jobs-in-bengaluru?k=data%20scientist&l=bengaluru&experience=0&nignbevent_src=jobsearchDeskGNB
Scraping page 1: https://www.naukri.com/data-scientist-jobs-in-bengaluru-1?
Scraping page 2: https://www.naukri.com/data-scientist-jobs-in-bengaluru-2?
